In [1]:
from sklearn.metrics.pairwise import linear_kernel
from sklearn.neighbors import NearestNeighbors
import numpy as np
from tqdm import tqdm_notebook
import pandas as pd
import seaborn as sns
import numpy as np
import warnings
from matplotlib.pyplot import imread
import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')
warnings.filterwarnings('ignore')
import codecs, json 

In [2]:
data=pd.read_csv('full_data.csv')

In [3]:
data.columns

Index(['заголовок', 'movieId', 'title', 'id', 'genres', 'cast', 'vote_average',
       'director', 'keywords', 'title_key', 'title_ru', 'directors_x1',
       'directors_x2', 'directors_cluster', 'genre_x1', 'genre_x2', 'genre_x3',
       'genre_cluster', 'cast_x1', 'cast_x2', 'cast_cluster', 'keywords_x1',
       'keywords_x2', 'keywords_cluster'],
      dtype='object')

In [4]:
X=data[['vote_average','directors_x1',
       'directors_x2', 'directors_cluster', 'genre_x1', 'genre_x2', 'genre_x3',
       'genre_cluster', 'cast_x1', 'cast_x2', 'cast_cluster', 'keywords_x1',
       'keywords_x2', 'keywords_cluster']]

In [5]:
X.head()

,vote_average,directors_x1,directors_x2,directors_cluster,genre_x1,genre_x2,genre_x3,genre_cluster,cast_x1,cast_x2,cast_cluster,keywords_x1,keywords_x2,keywords_cluster
0,7.7,2.915014,5.009351,0,-4.619838,-2.003358,-16.860966,9,-1.070758,-0.684697,16,12.052106,9.914565,1
1,6.6,6.993984,4.241862,1,17.111319,2.298493,-2.120147,17,1.537203,3.888912,11,-19.184790,6.082439,0
2,7.1,2.841570,31.452574,0,-6.898155,-6.148652,4.051120,7,-1.968244,0.360821,16,-3.689746,-0.817535,0
3,5.7,13.243169,1.783696,1,2.983787,-2.911626,-8.125521,18,1.869653,4.686978,1,20.795588,-6.392511,1
4,7.8,3.150442,31.063261,0,-13.161213,8.548437,4.870073,6,-3.203744,4.228349,14,6.337421,-11.446759,1


In [6]:
movies=data.заголовок

In [7]:
movies

0           История игрушек (1995)
1              Золотой глаз (1995)
2                    Никсон (1995)
3        Остров головорезов (1995)
4                    Казино (1995)
                   ...            
2532           Инфильтратор (2016)
2533         Плохие мамочки (2016)
2534                Площадь (2017)
2535                 Сияние (1997)
2536                Буйство (2018)
Name: заголовок, Length: 2537, dtype: object

In [37]:
top_films = data.groupby('заголовок')[['vote_average']].mean().sort_values('vote_average', ascending=False)
top_films.head(10)

,vote_average
заголовок,
"Танцовщица, Texas Pop. 81 (1998)",10.0
Криминальное чтиво (1994),8.3
Список Шиндлера (1993),8.3
Бойцовский клуб (1999),8.3
Энн из Зеленых Мезонинов (1985),8.2
Пролетая над гнездом кукушки (1975),8.2
Американская история X (1998),8.2
Славные парни (1990),8.2
Однажды в Америке (1984),8.2


In [38]:
# возьмем только фильмы с наивысшей средней оценкой в 5.0
films_with_highest_marks = top_films.iloc[np.where(top_films.vote_average >= 7.5)].index

In [39]:
# достанем по каждому фильму количество рейтингов
title_num_ratings = {}

for title, group in (data.groupby('заголовок')):
    title_num_ratings[title] = group.movieId.unique().shape[0]

In [40]:
len(films_with_highest_marks)

178

In [41]:
films_with_highest_marks

Index([' Танцовщица, Texas Pop. 81 (1998)', ' Криминальное чтиво (1994)',
       ' Список Шиндлера (1993)', ' Бойцовский клуб (1999)',
       ' Энн из Зеленых Мезонинов (1985)',
       ' Пролетая над гнездом кукушки (1975)',
       ' Американская история X (1998)', ' Славные парни (1990)',
       ' Однажды в Америке (1984)', ' Форрест Гамп (1994)',
       ...
       ' Что гложет Гилберта Грейпа (1993)',
       ' Гарри Поттер и Кубок огня (2005)', ' Маленькая мисс Счастье (2006)',
       ' Все еще Алиса (2014)', ' Крепкий орешек (1988)', ' Взвод (1986)',
       ' Челюсти (1975)', ' Лего-фильм (2014)', ' Контроль (2007)',
       ' Гаттака (1997)'],
      dtype='object', name='заголовок', length=178)

In [42]:
sorted([(title_num_ratings[f], f) for f in films_with_highest_marks], key=lambda x: x[0], reverse=True)[:10]

[(1, ' Танцовщица, Texas Pop. 81 (1998)'),
 (1, ' Криминальное чтиво (1994)'),
 (1, ' Список Шиндлера (1993)'),
 (1, ' Бойцовский клуб (1999)'),
 (1, ' Энн из Зеленых Мезонинов (1985)'),
 (1, ' Пролетая над гнездом кукушки (1975)'),
 (1, ' Американская история X (1998)'),
 (1, ' Славные парни (1990)'),
 (1, ' Однажды в Америке (1984)'),
 (1, ' Форрест Гамп (1994)')]

In [43]:
for i in range(len(movies)):
    if 'Бойцовский клуб (1999)' == movies[i]:
        print(i)

In [44]:
neigh = NearestNeighbors(n_neighbors=10, n_jobs=4, metric='manhattan') 
neigh.fit(X)

NearestNeighbors(metric='manhattan', n_jobs=4, n_neighbors=10)

In [45]:
res = neigh.kneighbors(X, return_distance=True)

In [46]:
for i in res[1][0]:
    print(movies[i])

 История игрушек (1995)
 История игрушек 2 (1999)
 История игрушек 3 (2010)
 Пчелиный фильм (2007)
 Полицейский из Беверли-Хиллз III (1994)
 Корпорация монстров (2001)
 Отель Трансильвания 2 (2015)
 Обманут! (2005)
 Первый фильм Дуга (1999)
 Любопытный Джордж (2006)


In [9]:
from sklearn.metrics.pairwise import cosine_distances
from sklearn.metrics.pairwise import manhattan_distances
from sklearn.metrics.pairwise import euclidean_distances
from sklearn.metrics.pairwise import laplacian_kernel

In [10]:
sim = laplacian_kernel(X, X)

In [11]:
sim.shape

(2537, 2537)

In [8]:
indices = pd.Series(data.index, index = data['заголовок']).drop_duplicates()
indices

заголовок
 История игрушек (1995)          0
 Золотой глаз (1995)             1
 Никсон (1995)                   2
 Остров головорезов (1995)       3
 Казино (1995)                   4
                              ... 
 Инфильтратор (2016)          2532
 Плохие мамочки (2016)        2533
 Площадь (2017)               2534
 Сияние (1997)                2535
 Буйство (2018)               2536
Length: 2537, dtype: int64

In [12]:
import re

In [13]:
data['Название'] = data['заголовок'].apply(lambda x: re.sub(r' \(\d{4}\)', '', x).strip())

In [14]:
data['Название']

0          История игрушек
1             Золотой глаз
2                   Никсон
3       Остров головорезов
4                   Казино
               ...        
2532          Инфильтратор
2533        Плохие мамочки
2534               Площадь
2535                Сияние
2536               Буйство
Name: Название, Length: 2537, dtype: object

In [15]:
indices = pd.Series(data.index, index = data['Название']).drop_duplicates()
indices

Название
История игрушек          0
Золотой глаз             1
Никсон                   2
Остров головорезов       3
Казино                   4
                      ... 
Инфильтратор          2532
Плохие мамочки        2533
Площадь               2534
Сияние                2535
Буйство               2536
Length: 2537, dtype: int64

In [16]:
indices.head()

Название
История игрушек       0
Золотой глаз          1
Никсон                2
Остров головорезов    3
Казино                4
dtype: int64

In [17]:
# def get_recommendation(title, sim = sim):
#     idx = indices[title]
#     sim_scores = enumerate(sim[idx])
#     sim_scores = sorted(sim_scores, key = lambda x: x[1], reverse = True)
#     sim_scores = sim_scores[1:11]
#     sim_index = [i[0] for i in sim_scores]
#     print(data["заголовок"].iloc[sim_index])

In [18]:
def get_recommendation(title, sim = sim):
    idx = indices[title]
    sim_scores = enumerate(sim[idx])
    sim_scores = sorted(sim_scores, key = lambda x: x[1], reverse = True)
    sim_scores = sim_scores[1:8]
    sim_index = [i[0] for i in sim_scores]
    print(data["заголовок"].iloc[sim_index])

In [19]:
get_recommendation('Чужой')

1932                            Начало (2010)
2288                      Интерстеллар (2014)
193                              Чужие (1986)
496                          ЭКСИСТЕНЗ (1999)
223      Звездный путь: Первый контакт (1996)
165                       Крайние меры (1996)
2121                            Линкор (2012)
Name: заголовок, dtype: object


In [4]:
pip install handler

Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install python-telegram-bot


  Obtaining dependency information for python-telegram-bot from https://files.pythonhosted.org/packages/d8/f0/b1c3f306d6846fb10d7bf737e2de010f33e43e682686e682b7cda93231a7/python_telegram_bot-21.4-py3-none-any.whl.metadata
   ---------------------------------------- 0.0/644.8 kB ? eta -:--:--
    --------------------------------------- 10.2/644.8 kB ? eta -:--:--
   - ------------------------------------- 30.7/644.8 kB 445.2 kB/s eta 0:00:02
   ---- ---------------------------------- 71.7/644.8 kB 563.7 kB/s eta 0:00:02
   -------- ----------------------------- 143.4/644.8 kB 853.3 kB/s eta 0:00:01
   ------------- -------------------------- 225.3/644.8 kB 1.1 MB/s eta 0:00:01
   ----------------- ---------------------- 286.7/644.8 kB 1.1 MB/s eta 0:00:01
   ---------------------- ----------------- 368.6/644.8 kB 1.3 MB/s eta 0:00:01
   --------------------------------- ------ 542.7/644.8 kB 1.6 MB/s eta 0:00:01
   ---------------------------------------- 644.8/644.8 kB 1.7 MB/s eta 0: